In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

def fetch_realtime_data():
    # Initialize the webdriver
    chrome_service = Service(r'C:\Users\user\chromedriver-win64\chromedriver.exe')
    driver = webdriver.Chrome(service=chrome_service)
   
    # Open the webpage
    url = "https://epsis.kpx.or.kr/epsisnew/selectEkgeEpsMepRealChart.do?menuId=030300"
    driver.get(url)
    
    # Maximize the window
    driver.maximize_window()
    
    # Scroll to the bottom of the page to load initial data
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    
    # Initialize an empty DataFrame to store all data elements
    df = pd.DataFrame(columns=['temporary', 'supply capacity', 'current load', 'supply reserve capacity', 'supply reserve rate'])
    
    # Find the scrollbar element
    scrollbar = driver.find_element(By.CLASS_NAME, 'rMateH5__VBrowserScrollBar')
    
    scroll_count = 0
    max_scrolls = 24  # Set a maximum number of scrolls to prevent infinite loop
    
    while True:
        # Get the initial number of elements
        initial_size = len(df)

        # Get the page source and parse it
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Find elements containing the data
        elements = soup.find_all(class_=lambda c: c and c.startswith('rMateH5__DataGridItemRenderer rMateH5__DataGridColumn'))
        
        # Extract text from elements and add to a temporary list if not null
        new_elements = [element.text for element in elements if element.text.strip()]
        
        # Get the initial scrollbar position
        initial_scroll_position = driver.execute_script("return arguments[0].scrollTop;", scrollbar)
        
        # Scroll the scrollbar down a small amount
        driver.execute_script("arguments[0].scrollTop += 580;", scrollbar)
        time.sleep(2)
        
        # Get the new scrollbar position
        new_scroll_position = driver.execute_script("return arguments[0].scrollTop;", scrollbar)

        for element in new_elements:
            print(element)
        
        # Split the data into respective categories and append to the DataFrame
        if new_elements:
            num_entries = len(new_elements)
            num_iter = num_entries // 5 * 4   

            supply_capacity = []
            current_load = []
            supply_reserve_capacity = []
            supply_reserve_rate = []
            
            
            for i in range(0,num_iter,4):
                supply_capacity.append(new_elements[i])
                current_load.append(new_elements[i+1])
                supply_reserve_capacity.append(new_elements[i+2])
                supply_reserve_rate.append(new_elements[i+3])
            
            timestamps = new_elements[num_iter:num_entries]
                
            
            
            temp_df = pd.DataFrame({
                'temporary': timestamps,
                'supply capacity': supply_capacity,
                'current load': current_load,
                'supply reserve capacity': supply_reserve_capacity,
                'supply reserve rate': supply_reserve_rate
            })
            
            df = pd.concat([df, temp_df], ignore_index=True)
        
        # Check if scrollbar position did not change
        if initial_scroll_position == new_scroll_position:
            break
        
        # Increment scroll counter and check if max scrolls reached
        scroll_count += 1
        if scroll_count >= max_scrolls:
            print("Reached maximum scroll limit.")
            break
    
    # Close the webdriver
    driver.quit()
    
    # Print the DataFrame
    df = df.drop_duplicates()
    df = df.sort_values(by = 'temporary')

    df.to_excel('today.xlsx', index=False)

In [3]:
fetch_realtime_data()

85,975
62,454
21,564
34
85,953
61,926
22,067
35
85,950
61,297
22,689
36
85,928
61,156
22,808
36
85,860
60,845
23,052
37
85,827
60,781
23,078
37
85,790
60,833
22,992
37
85,748
61,091
22,688
36
85,741
61,025
22,747
36
85,676
61,220
22,490
36
85,646
61,384
22,295
35
85,607
61,393
22,253
35
85,668
61,189
22,863
36
85,638
61,458
22,601
36
85,613
62,250
22,108
35
85,557
62,559
21,742
34
2024-06-10 13:00
2024-06-10 12:55
2024-06-10 12:50
2024-06-10 12:45
2024-06-10 12:40
2024-06-10 12:35
2024-06-10 12:30
2024-06-10 12:25
2024-06-10 12:20
2024-06-10 12:15
2024-06-10 12:10
2024-06-10 12:05
2024-06-10 12:00
2024-06-10 11:55
2024-06-10 11:50
2024-06-10 11:45
85,432
62,990
21,184
33
85,347
63,014
21,074
33
85,311
63,146
20,905
33
85,216
63,139
20,820
32
85,184
63,349
20,582
32
85,124
63,195
20,671
32
85,072
63,246
20,577
32
85,751
63,582
20,908
32
85,739
63,647
20,837
32
85,698
63,952
20,492
31
85,668
63,778
20,635
32
85,340
63,853
20,237
31
85,359
63,870
20,232
31
85,250
63,824
20,168
31
85,204
6